In [ ]:
import requests
import json
import pandas as pd
import time
import schedule
import xlwings as xw


def job():
    url = 'https://www.nseindia.com/api/option-chain-indices?symbol=NIFTY'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36 Edg/112.0.1722.68'
    }

    session = requests.Session()
    response = session.get(url, headers=headers, timeout=10)
    
    response_text = response.text
    json_object = json.loads(response_text)

    with open("OC.json", "w") as outfile:
        outfile.write(response_text)

    data = json_object['records']['data']
    e_date = json_object['records']['expiryDates']

    oc_data = {}
    for ed in e_date:
        oc_data[ed] = {"CE": [], "PE": []}
        for di in range(len(data)):
            if data[di]['expiryDate'] == ed:
                if 'CE' in data[di].keys() and data[di]['CE']['expiryDate'] == ed:
                    oc_data[ed]["CE"].append(data[di]['CE'])
                else:
                    oc_data[ed]["CE"].append('-')

                if 'PE' in data[di].keys() and data[di]['PE']['expiryDate'] == ed:
                    oc_data[ed]["PE"].append(data[di]['PE'])
                else:
                    oc_data[ed]["PE"].append('-')

    for k in oc_data.keys():
        for i in range(len(oc_data[k]["CE"])):
            if oc_data[k]["CE"][i] != '-':
                del oc_data[k]["CE"][i]["expiryDate"]
                del oc_data[k]["CE"][i]["underlying"]
                del oc_data[k]["CE"][i]["identifier"]

            if oc_data[k]["PE"][i] != '-':
                del oc_data[k]["PE"][i]["expiryDate"]
                del oc_data[k]["PE"][i]["underlying"]
                del oc_data[k]["PE"][i]["identifier"]

    oc_data_dt = oc_data['28-Dec-2023']
    CE = list(oc_data_dt['CE'])
    PE = list(oc_data_dt['PE'])
    
    df = pd.DataFrame(oc_data_dt)
    wb = xw.Book("OC-Data.xlsx")
    st = wb.sheets["NIFTY"]
    st.range("A1").value = df


# Schedule the job to run every 3 minutes
schedule.every(3).minutes.do(job)

# Run the scheduled jobs indefinitely
while True:
    schedule.run_pending()
    time.sleep(1)
